In [1]:
import pandas as pd

import numpy as np

import string

from nltk.stem import PorterStemmer

from nltk.tokenize import RegexpTokenizer

from nltk.corpus import stopwords
from statsmodels.graphics.mosaicplot import mosaic

import matplotlib.pyplot as plt

import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
!pip install modelscope

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 69.3 MB/s eta 0:00:00


In [3]:
from modelscope import snapshot_download
model_id = snapshot_download("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

2025-03-11 22:57:59,219 - modelscope - INFO - Got 9 files, start to download ...


Processing 9 items:   0%|          | 0.00/9.00 [00:00<?, ?it/s]

2025-03-11 22:59:27,455 - modelscope - INFO - Download model 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B' successfully.
2025-03-11 22:59:27,456 - modelscope - INFO - Creating symbolic link [/root/.cache/modelscope/hub/models/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B].


In [4]:
from transformers import pipeline

In [5]:
deepseek_pipeline = pipeline(
    "text-generation",
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    device="cuda",
    model_kwargs={"torch_dtype": "auto"}  
)


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Device set to use cuda


In [6]:
test_filepath_csv = '../../data/processed/test.csv'

In [7]:
test_df = pd.read_csv(test_filepath_csv)
test_df

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Unnamed: 0,premise,hypothesis,gold_label,type
0,9616.0,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,type5_possessive
1,9617.0,It's not the case that if Charlotte passes her...,Charlotte has parents.,E,type5_possessive
2,9618.0,Sam believes that if Charlotte passes her driv...,Charlotte has parents.,E,type5_possessive
3,9619.0,Is it true that if Charlotte passes her drivin...,Charlotte has parents.,E,type5_possessive
4,9628.0,"If Lily takes the train to the city, her frien...",Lily has a friend.,E,type5_possessive
...,...,...,...,...,...
2362,NaN,"If Sophia decides to attend the concert, his b...",Sophia has a close friend.,N,type5_possessive
2363,NaN,It's not the case that if Sophia decides to at...,Sophia has a close friend.,N,type5_possessive
2364,NaN,Sam believes that if Sophia decides to attend ...,Sophia has a close friend.,N,type5_possessive
2365,NaN,Is it true that if Sophia decides to attend th...,Sophia has a close friend.,N,type5_possessive


In [8]:
# Split the DataFrame into two halves
half = len(test_df) // 2
test_df1 = test_df[:half]
test_df2 = test_df[half:]

In [9]:
test_df1

,Unnamed: 0,premise,hypothesis,gold_label,type
0,9616.0,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,type5_possessive
1,9617.0,It's not the case that if Charlotte passes her...,Charlotte has parents.,E,type5_possessive
2,9618.0,Sam believes that if Charlotte passes her driv...,Charlotte has parents.,E,type5_possessive
3,9619.0,Is it true that if Charlotte passes her drivin...,Charlotte has parents.,E,type5_possessive
4,9628.0,"If Lily takes the train to the city, her frien...",Lily has a friend.,E,type5_possessive
...,...,...,...,...,...
1178,2274.0,Is it true that if Amy ever bought a smartphon...,Amy has bought an iPhone.,N,type3_again
1179,2280.0,It's not the case that if the president ever s...,The president has signed the ceasefire agreement.,N,type3_again
1180,2285.0,It's not the case that if the president ever p...,The president has participated in the Rolling ...,N,type3_again
1181,2287.0,It's not the case that if the old man ever fed...,The old man has fed a snow leopard.,N,type3_again


In [10]:
test_df2

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Unnamed: 0,premise,hypothesis,gold_label,type
1183,2294.0,Sam believes that if John ever trusted a condu...,John has trusted Leonard Bernstein.,N,type3_again
1184,2296.0,"Sam believes that if Amy ever pet a parrot, sh...",Amy has pet a Macaw.,N,type3_again
1185,2297.0,It's not the case that if Carlos ever watched ...,Carlos has watched Long Walk to Freedom.,N,type3_again
1186,2299.0,"If the woman ever studied a genre, she didn't ...",The woman has studied science fiction.,N,type3_again
1187,2306.0,Sam believes that if the professor ever used a...,The professor has used Pointillism.,N,type3_again
...,...,...,...,...,...
2362,NaN,"If Sophia decides to attend the concert, his b...",Sophia has a close friend.,N,type5_possessive
2363,NaN,It's not the case that if Sophia decides to at...,Sophia has a close friend.,N,type5_possessive
2364,NaN,Sam believes that if Sophia decides to attend ...,Sophia has a close friend.,N,type5_possessive
2365,NaN,Is it true that if Sophia decides to attend th...,Sophia has a close friend.,N,type5_possessive


In [11]:
def classify_pair(row):
    premise = row["premise"]
    hypothesis = row["hypothesis"]
    messages = [
        {"role": "user", "content": f'''
        Does the premise entail the hypothesis? 
        Output E if the premise does entails the hypothesis, or N if it does not. 
        Output C if the hypothesis contradicts the premise. Do not output anything other than E, N, or C.
        Premise: {premise}
        Hypothesis: {hypothesis}
        Output:
        '''},
    ]
    
    outputs = deepseek_pipeline(
        messages,
        max_new_tokens=2000,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    
    return outputs[0]["generated_text"][-1]['content'][-1]

In [12]:
[classify_pair(test_df1.iloc[-1*i]) for i in range(3)]

['N', 'E', 'N']

In [13]:
test_df1['Predicted - Zero-Shot DeepSeek'] = test_df1.apply(classify_pair, axis=1)
test_df1.to_csv('deepseek_test_df1_zeroshot_results.csv')

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
<ipython-input-13-87a74319d396>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df1['Predicted - Zero-Shot DeepSeek'] = test_df1.apply(classify_pair, axis=1)
